In [10]:
import json
from pyspark import SparkContext
from time import time

In [2]:
path = "/Users/liangsiqi/Documents/Dataset/yelp_dataset/"
user_file = "user.json"
review_file = "review.json"
business_file = "business.json"

In [3]:
def run():
    sc = SparkContext()
    reviewRDD = sc.textFile(path+review_file).map(json.loads)
    businessRDD = sc.textFile(path+business_file).map(json.loads)

    business_state_kvRDD = businessRDD.map(lambda x: [x['business_id'],x['state']])
    business_id_star_kvRDD = reviewRDD.map(lambda x: [x['business_id'],x['stars']])
    business_id_star_combine = business_id_star_kvRDD.combineByKey((lambda x: (x,1)),  # ceateCombiner
                                (lambda x,value: (x[0] + value, x[1] + 1)),  # mergeValue
                                (lambda x,y: (x[0] + y[0], x[1] + y[1])))
    interm_state_stars_cnt_combine = business_id_star_combine.join(business_state_kvRDD).map(lambda x: (x[1][1],x[1][0]))
    interm_state_stars_cnt_combine.persist()
    interm_state_stars_cnt = interm_state_stars_cnt_combine.combineByKey((lambda x: (x,1)),  # ceateCombiner
                                        (lambda c,value: ((c[0][0] + value[0], c[0][1] + value[1]), c[1]+1)),  # mergeValue
                                        (lambda x,y: ((x[0][0] + y[0][0], x[0][1] + y[0][1]), x[1] + y[1])))
    avg_state = interm_state_stars_cnt.map(lambda x: [x[0], x[1][0][0] / x[1][0][1]]).sortBy(lambda x: (-x[1],x[0]))
    interm_state_stars_cnt_combine.unpersist()
    result = avg_state.collect()
    sc.stop()
    return result

def run2():
    sc = SparkContext()
    reviewRDD = sc.textFile(path+review_file).map(json.loads)
    businessRDD = sc.textFile(path+business_file).map(json.loads)

    business_state_kvRDD = businessRDD.map(lambda x: [x['business_id'],x['state']])
    business_id_star_kvRDD = reviewRDD.map(lambda x: [x['business_id'],x['stars']])
    business_id_star_combine = business_id_star_kvRDD.combineByKey((lambda x: (x,1)),  # ceateCombiner
                                (lambda x,value: (x[0] + value, x[1] + 1)),  # mergeValue
                                (lambda x,y: (x[0] + y[0], x[1] + y[1])))
    interm_state_stars_cnt_combine = business_id_star_combine.join(business_state_kvRDD).map(lambda x: (x[1][1],x[1][0]))
    interm_state_stars_cnt = interm_state_stars_cnt_combine.combineByKey((lambda x: (x,1)),  # ceateCombiner
                                        (lambda c,value: ((c[0][0] + value[0], c[0][1] + value[1]), c[1]+1)),  # mergeValue
                                        (lambda x,y: ((x[0][0] + y[0][0], x[0][1] + y[0][1]), x[1] + y[1])))
    avg_state = interm_state_stars_cnt.map(lambda x: [x[0], x[1][0][0] / x[1][0][1]]).sortBy(lambda x: (-x[1],x[0]))
    result = avg_state.collect()
    sc.stop()
    return result

def run3():
    sc = SparkContext()
    reviewRDD = sc.textFile(path+review_file).map(json.loads)
    businessRDD = sc.textFile(path+business_file).map(json.loads)

    business_state_kvRDD = businessRDD.map(lambda x: [x['business_id'],x['state']])
    business_id_star_kvRDD = reviewRDD.map(lambda x: [x['business_id'],x['stars']])
    business_id_star_combine = business_id_star_kvRDD.combineByKey((lambda x: (x,1)),  # ceateCombiner
                                (lambda x,value: (x[0] + value, x[1] + 1)),  # mergeValue
                                (lambda x,y: (x[0] + y[0], x[1] + y[1])))
    interm_state_stars_cnt_combine = business_id_star_combine.join(business_state_kvRDD).map(lambda x: (x[1][1],x[1][0]))
    interm_state_stars_cnt_combine.persist()
    interm_state_stars_cnt = interm_state_stars_cnt_combine.combineByKey((lambda x: (x,1)),  # ceateCombiner
                                        (lambda c,value: ((c[0][0] + value[0], c[0][1] + value[1]), c[1]+1)),  # mergeValue
                                        (lambda x,y: ((x[0][0] + y[0][0], x[0][1] + y[0][1]), x[1] + y[1])))
    avg_state = interm_state_stars_cnt.map(lambda x: [x[0], x[1][0][0] / x[1][0][1]]).sortBy(lambda x: (-x[1],x[0]))
    result = avg_state.collect()
    interm_state_stars_cnt_combine.unpersist()
    sc.stop()
    return result

def run4():
    sc = SparkContext()
    reviewRDD = sc.textFile(path+review_file).map(json.loads)
    businessRDD = sc.textFile(path+business_file).map(json.loads)
    business_state_kvRDD = businessRDD.map(lambda x: [x['business_id'],x['state']])
    business_id_star_kvRDD = reviewRDD.map(lambda x: [x['business_id'],x['stars']])
    join_state_stars = business_id_star_kvRDD.join(business_state_kvRDD).map(lambda x: [x[1][1],x[1][0]])
    state_star_combine = join_state_stars.combineByKey((lambda x: (x,1)),  # ceateCombiner
                                (lambda x,value: (x[0] + value, x[1] + 1)),  # mergeValue
                                (lambda x,y: (x[0] + y[0], x[1] + y[1])))
    avg_state = state_star_combine.map(lambda x: [x[0], x[1][0]/x[1][1]]).sortBy(lambda x: (-x[1],x[0]))
    result = avg_state.collect()
    sc.stop()
    return result

In [4]:
print("run():")
%timeit run()

print("\nrun2():")
%timeit run2()

print("\nrun3():")
%timeit run3()

print("\nrun4():")
%timeit run4()

run():
24.8 s ± 101 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

run2():
24.3 s ± 183 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

run3():
24.6 s ± 114 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

run4():
26.3 s ± 170 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
sc = SparkContext.getOrCreate()
kv1 = sc.parallelize([('a',1),
                      ('b',2),
                      ('c',3),
                      ('c',4),
                      ('d',5),
                      ('d',6),
                      ('a',7),
                      ('e',8)])
kv2 = sc.parallelize([('a','ca'),
                      ('b','ca'),
                      ('c','ny'),
                      ('d','fl'),
                      ('e','ny')])

In [6]:
kv1.join(kv2).collect()
sc.stop()

[('b', (2, 'ca')),
 ('d', (5, 'fl')),
 ('d', (6, 'fl')),
 ('a', (1, 'ca')),
 ('a', (7, 'ca')),
 ('e', (8, 'ny')),
 ('c', (3, 'ny')),
 ('c', (4, 'ny'))]

In [5]:
resultA = run2()

with open("task3A.txt", "w") as outA:
    outA.write("state,stars\n")
    for iterm in resultA:
        outA.write("%s,%.3f\n" % (iterm[0], iterm[1]))

In [6]:
for i in range(5):
    print(resultA[i][0])

TN
NJ
VT
XWY
XGL


In [11]:
sc = SparkContext()
reviewRDD = sc.textFile(path+review_file).map(json.loads)
businessRDD = sc.textFile(path+business_file).map(json.loads)

business_state_kvRDD = businessRDD.map(lambda x: [x['business_id'], x['state']])
business_id_star_kvRDD = reviewRDD.map(lambda x: [x['business_id'], x['stars']])
business_id_star_combine = business_id_star_kvRDD.combineByKey((lambda x: (x, 1)),  # ceateCombiner
                                                               (lambda x, value: (x[0] + value, x[1] + 1)),  # mergeValue
                                                               (lambda x, y: (x[0] + y[0], x[1] + y[1])))

interm_state_stars_cnt_combine = business_id_star_combine.join(business_state_kvRDD).map(lambda x: (x[1][1], x[1][0]))

interm_state_stars_cnt = interm_state_stars_cnt_combine.combineByKey((lambda x: (x, 1)),  # ceateCombiner
                                                                     (lambda c, value: ((c[0][0] + value[0], c[0][1] + value[1]),
                                                                                        c[1] + 1)),  # mergeValue
                                                                     (lambda x, y: ((x[0][0] + y[0][0], x[0][1] + y[0][1]),
                                                                                    x[1] + y[1])))
avg_state = interm_state_stars_cnt.map(lambda x: [x[0], x[1][0][0] / x[1][0][1]]).sortBy(lambda x: (-x[1], x[0]))
print("Methd 1:")
start1 = time()
resultA = avg_state.collect()
for i in range(5):
    print(resultA[i][0])
end1 = time()
m1 = end1 - start1

print("Methd 2:")
start2 = time()
resultB = avg_state.take(5)
for i in range(5):
    print(resultB[i][0])
end2 = time()
m2 = end2 - start2

sc.stop()

print("Method 1: %.10f" % m1)
print("Method 2: %.10f" % m2)

Methd 1:
TN
NJ
VT
XWY
XGL
Methd 2:
TN
NJ
VT
XWY
XGL
Method 1: 0.5144200325
Method 2: 0.0914618969
